In [1]:
import os
import random
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


In [2]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

CUDA available: True
Number of GPUs: 1
GPU name: NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
import os
import random
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# -----------------
# Dataset Class
# -----------------
class SignaturePairsDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        root_dir: path to your 'dataset' folder
        transform: torchvision transforms to apply to images
        """
        self.root_dir = root_dir
        self.transform = transform

        # Collect person IDs (exclude forged folders)
        self.people = [
            d for d in os.listdir(root_dir) 
            if os.path.isdir(os.path.join(root_dir, d)) and not d.endswith("forg")
        ]
        self.people.sort()
        print(f"Found {len(self.people)} people.")

        # Map person_id to their genuine and forged images
        self.data = {}
        for pid in self.people:
            genuine_dir = os.path.join(root_dir, pid)
            forged_dir = os.path.join(root_dir, pid + " forg")
            
            genuine_images = [os.path.join(genuine_dir, f) for f in os.listdir(genuine_dir)]
            forged_images = [os.path.join(forged_dir, f) for f in os.listdir(forged_dir)] if os.path.exists(forged_dir) else []

            if len(genuine_images) == 0:
                print(f"Warning: {pid} has no genuine images. Skipping.")
                continue

            self.data[pid] = {
                "genuine": genuine_images,
                "forged": forged_images
            }

        # Remove people with no data
        self.people = list(self.data.keys())

    def __len__(self):
        # Arbitrary large number to generate enough pairs
        return 100000

    def __getitem__(self, idx):
        # Randomly choose a person
        person_id = random.choice(self.people)
        genuine_images = self.data[person_id]["genuine"]
        forged_images = self.data[person_id]["forged"]

        # Decide positive or negative pair
        if forged_images and random.random() < 0.5:
            # Negative pair: genuine vs forged
            img1_path = random.choice(genuine_images)
            img2_path = random.choice(forged_images)
            label = 0
        else:
            # Positive pair: two genuine signatures
            if len(genuine_images) < 2:
                img1_path = img2_path = genuine_images[0]
            else:
                img1_path, img2_path = random.sample(genuine_images, 2)
            label = 1

        # Load images
        img1 = Image.open(img1_path).convert("L")
        img2 = Image.open(img2_path).convert("L")

        # Apply transforms if any
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, torch.tensor([label], dtype=torch.float32)


# -----------------
# Define transform (resize + normalize)
# -----------------
transform = transforms.Compose([
    transforms.Resize((128, 128)),      # compress to 128x128
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# -----------------
# Create dataset + dataloader
# -----------------
dataset = SignaturePairsDataset("dataset", transform=transform)
dataloader = DataLoader(dataset, shuffle=True, batch_size=16)


Found 23 people.


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SiameseNetwork(nn.Module):
    def __init__(self, input_size=(1, 128, 128)):
        super(SiameseNetwork, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )

        # Compute flattened size dynamically
        with torch.no_grad():
            dummy_input = torch.zeros(1, *input_size)
            dummy_output = self.cnn(dummy_input)
            flattened_size = dummy_output.view(1, -1).size(1)

        self.fc = nn.Sequential(
            nn.Linear(flattened_size, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 128)
        )

    def forward_once(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    def forward(self, x1, x2):
        out1 = self.forward_once(x1)
        out2 = self.forward_once(x2)
        return out1, out2


In [5]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss = torch.mean(
            (1 - label) * torch.pow(euclidean_distance, 2) +
            label * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2)
        )
        return loss


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork().to(device)
criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)


In [12]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for img1, img2, labels in dataloader:
        img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)

        optimizer.zero_grad()
        out1, out2 = model(img1, img2)
        loss = criterion(out1, out2, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

torch.save(model.state_dict(), "siamese_signature.pth")


Epoch [1/10], Loss: 0.0013
Epoch [2/10], Loss: 0.0000
Epoch [3/10], Loss: 0.0000
Epoch [4/10], Loss: 0.0000
Epoch [5/10], Loss: 0.0000
Epoch [6/10], Loss: 0.0000
Epoch [7/10], Loss: 0.0000
Epoch [8/10], Loss: 0.0000
Epoch [9/10], Loss: 0.0000
Epoch [10/10], Loss: 0.0000


In [7]:
test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
